In [7]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains 
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException     
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary

from transformers import BertTokenizer, BertForSequenceClassification
from torch.nn.functional import softmax
from konlpy.tag import Okt
from tqdm import tqdm 

import tensorflow as tf 
import torch
import time
import numpy as np
import random
import pandas as pd
from pandas import json_normalize
import time
from datetime import datetime, timezone
import csv
import random
import re
import json
import traceback
import requests

In [8]:
options = Options()
options.add_experimental_option("detach", True)
#클라이언트의 식별정보를 담은 헤더: 이거 추가해서 넣으면 봇으로 탐지 안 될 수 있음. 
#https://www.whatismybrowser.com/detect/what-is-my-user-agent/
options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/119.0') 
# navigator.webdriver 탐지 피하기 
# f12 > console > navigator.webdriver 입력 > false로 나오면 화면 접속 중인 client가 봇인지 아닌지 판별
# 웹사이트마다 정책이 달라서 사용할 수 있는 보조 수단으로 생각
options.add_argument('--disable-blink-features=AutomationControlled') 

In [9]:
#리뷰 저장할 리스트 만들기 
review_list = []
#에러 목록 저장할 리스트 만들기: 에러 생기는 값들 저장해두기 
error_list = []

In [10]:
# 여러 webdriver 사용 가능한데 chrome이 메모리 많이 잡아먹음 -> 데이터가 만 단위 넘어가면
# 브라우저도 메모리가 있어서 돌다가 안 돌아감
# 파이어폭스 브라우저가 메모리 사용량이 더 적음 
# browser = webdriver.Chrome()
browser = webdriver.Firefox()

#네이버쇼핑 url 열기 
url = 'https://shopping.naver.com/home'
browser.get(url)
time.sleep(random.uniform(2,3)) # 잠시 멈추기 위한 장치

#수집할 상품 키워드 
product_keyword = '롱패딩'

#검색어 입력 
# browser.find_element(By.XPATH,'Xpath경로'): 우리가 연 브라우저에서 XPATH로 element를 찾는다는 뜻
# f12 > 화살표 > 검색창에 갖다대고 > html에서 > copy > copy xpath
# By.XPATH: XPATH로 찾겠다는 의미
searchbox = browser.find_element(By.XPATH,'//*[@id="gnb-gnb"]/div[2]/div/div[2]/div/div[2]/form/div[1]/div/input').send_keys(product_keyword)

#돋보기 클릭 //*[@id="gnb-gnb"]/div/div[1]/button/span
search_icon = browser.find_element(By.XPATH,'//*[@id="gnb-gnb"]/div[2]/div/div[2]/div/div[2]/form/div[1]/div/button[2]')
search_icon.click()

In [11]:
# 맨 하단까지 스크롤해서 전체정보 다 가져오기 (동적 웹사이트)
# https://stackoverflow.com/questions/20986631/how-can-i-scroll-a-web-page-using-selenium-webdriver-in-python
# Get scroll height / 스크롤 전 스크롤의 위치 
last_height = browser.execute_script("return document.body.scrollHeight")

while True:
    # 현재 화면에서 스크롤 가장 아래로 내림 
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    # 새로운 페이지 로딩 될때까지 기다리기 
    time.sleep(random.uniform(1.5,2))
    # 새로운 스크롤 높이 변수로 저장 
    new_height = browser.execute_script("return document.body.scrollHeight")    
    # print(last_height,'/',new_height)    
    # 새로운 스크롤 위치와 이동 전 스크롤 위치 같으면(더 이상 스크롤이 늘어나지 않으면) 종료
    if new_height == last_height:
        break
    last_height = new_height

#전체 상품 수
all_pdt_num = browser.find_element(By.XPATH,'//*[@id="content"]/div[1]/div[1]/ul/li[1]/a/span[1]').text
print('전체 상품 수 : ',all_pdt_num)

review_button = browser.find_element(By.XPATH, '//*[@id="content"]/div[1]/div[2]/div/div[4]/div/div/div[2]/div[5]/a/em')
review_button.click()

전체 상품 수 :  1,465,420


In [12]:
# --------------------------------- 여러 상품의 리뷰 수집하기  --------------------------------- #
for pdt_page in range(1,3): #임시로 3페이지까지. 전체 페이지 다 긁을거면 all_pdt_num 활용 
    print( '<================= ', '리뷰 페이지 : ',pdt_page, ' =================>')
    time.sleep(random.uniform(2,3))
# --------------------------------- 한 페이지의 리뷰 다 가져오기 --------------------------------- #
    for pdt_num in range(4,47):
        time.sleep(random.uniform(3,4))
        try: 
            xpath_1 = '//*[@id="content"]/div[1]/div[2]/div/div[{0}]/div/div/div[3]/div/a'
            name_tag = browser.find_element(By.XPATH,xpath_1.format(pdt_num)).text
            try : # 어디서 에러 발생할 지 모르니까 전체 try-except 걸기
                # 쇼핑몰별 최저가 텍스트로 조건 걸기 -> {0}으로 순서 표시
                # 이걸로 시도해보자: try문 걸기 (연결 페이지 구조 다르면 작동 안 됨)
                if name_tag == '쇼핑몰별 최저가':  #'쇼핑몰별 최저가' 링크가 아닌 기타 링크일 페이지 구조가 다르므로 스킵
                    review_button = browser.find_element(By.XPATH,'//*[@id="content"]/div[1]/div[2]/div/div[{0}]/div/div/div[2]/div[5]/a/em'.format(pdt_num))
                    review_button.click()
                    '''
                    # div[1],[2],[3] 은 광고 상품이라 4부터 시작함.
                    //*[@id="content"]/div[1]/div[2]/div/div[4]/div/div/div[2]/div[5]/a/em #페이지 가장 위 첫 상품 
                    //*[@id="content"]/div[1]/div[2]/div/div[5]/div/div/div[2]/div[5]/a/em
                    //*[@id="content"]/div[1]/div[2]/div/div[6]/div/div/div[2]/div[5]/a/em
                    //*[@id="content"]/div[1]/div[2]/div/div[7]/div/div/div[2]/div[5]/a/em #페이지 가장 아래 마지막 상품 
                    '''
                    # if문 성공했을시:
                    # --------------------------------- 상품 하나의 리뷰 가져오기 --------------------------------- #
                    ### !새 탭으로 이동 (리뷰 누르면 새로운 창으로 열림) 
                    time.sleep(random.uniform(3,4))
                    browser.switch_to.window(browser.window_handles[1])
                    
                    # 상품 기본정보 불러오기 - 여기도 에러 방지 위해 try문 입력
                    # 근데 xpath 어디서 가져온 건지 잘 모르겠음..
                    try:
                        pdt_name = browser.find_element(By.XPATH,'/html/body/div/div/div[2]/div[2]/div[1]/h2').text #상품명
                    except :
                        None
                    try:
                        upload_date = browser.find_element(By.XPATH,'/html/body/div/div/div[2]/div[2]/div[1]/div[2]/span[2]/em').text #등록일
                    except:
                        None
                    
                    print('-------------------------------------------------')
                    print('상품명 : ', pdt_name ,'/ 등록일 : ', upload_date)
                    

                    ### !리뷰 가져오기 
                    ## 총 리뷰수
                    # 페이지에 해당 숫자를 찾을 수 있는 곳이 여러 군데 있는데, 숫자를 분리하지 않고 통으로 가져올 수 있는 것 선택
                    total_review_num = browser.find_element(By.XPATH,'//*[@id="section_review"]/div[2]/div[2]/ul/li[1]/a/em')
                    total_review_num_text = re.sub(r'[^0-9]', '' ,total_review_num.text ) #정규식으로 ( ) 제거: 숫자 제외한 나머지 값들 전부 제거 
                    print("총리뷰수 : ", total_review_num_text) #중간중간 print 해주는게 좋음 
                    
                    
                    ## 리뷰 마지막 페이지: 리뷰/페이지 수 해서 나머지 생기면 + 1 해서 남은 리뷰 수도 돌아가게 
                    if int(total_review_num_text) % 20 == 0: # 딱 나눠 떨어지면 
                        last_page = (int(total_review_num_text)//20) #총 리뷰 나누기 (몫만큼 알고리즘 돌리면 되니까) 
                    else:
                        last_page = (int(total_review_num_text)//20) + 1 #총 리뷰 나누기 20 + 1 (나머지 리뷰 모으러 한 번 더 돌아가게)
                    print("마지막페이지 : ", last_page)
                    
                    #한 상품 내에서 전체 페이지 돌아가면서 리뷰 수집하기 
                    for page in range(1, last_page + 1 ): # for loop에서는 마지막 페이지 + 1
                        time.sleep(random.uniform(2,3))
                        print('{0} 페이지'.format(page)) # for문 돌아가면서 1 페이지, 2페이지, ... 이렇게 print됨
                        
                        #각 페이지 내의 리뷰 20개 가져오기 (1페이지의 20개)
                        for i in range(1,21):

                            try:
                                별점 = browser.find_element(By.XPATH,'//*[@id="section_review"]/ul/li[{0}]/div[1]/span[1]/span/span'.format(i)).get_attribute("style")
                                별점_num = int(re.findall(r'\d+', 별점)[0])//20
                                구매처 = browser.find_element(By.XPATH,'//*[@id="section_review"]/ul/li[{0}]/div[1]/span[2]'.format(i)).text
                                날짜 = browser.find_element(By.XPATH,'//*[@id="section_review"]/ul/li[{0}]/div[1]/span[4]'.format(i)).text
                                리뷰 = browser.find_element(By.XPATH,'//*[@id="section_review"]/ul/li[{0}]/div[2]/div[1]/p'.format(i)).text
                            except:
                                별점 = None
                                구매처 = None
                                날짜 = None
                                리뷰 = None
                            #개별 리뷰의 각 요소를 빈 딕셔너리에 넣기 
                            info_dict = {}
                            info_dict['pdt_name'] = pdt_name 
                            info_dict['rating'] = 별점_num
                            info_dict['store'] = 구매처
                            info_dict['date'] = 날짜
                            info_dict['review'] = 리뷰
                            #info_dict를 리뷰 리스트에 추가 --> 나중에 한번에 데이터프레임으로 바꾸기 
                            review_list.append(info_dict) 
                        
                        time.sleep(random.uniform(1,2)) # 각 페이지별로 로딩 시간 걸어주기 
                        
                        # 앞서 한 페이지 완료했으니 이제 다음 페이지 버튼 클릭 
                        try: # ex) 마지막 페이지가 9인 경우에는, //*[@id="section_review"]/div[3]/a[10] 가 없어서 코드 에러가 발생하므로 에러 넘기기 
                            #10 페이지에서는 '다음' 버튼을 눌러야 함 
                            if page == 10 : #리뷰 화면에서 첫 페이지와, 2번째 페이지부터의 '다음'의 xpath가 다름 : 첫페이지는 a[11]
                                next_button = browser.find_element(By.XPATH,'//*[@id="section_review"]/div[3]/a[11]')
                                next_button.click()
                                # print('1번옵션')
                            #20 페이지 이상부터 10단위마다 '다음' 누르기     
                            elif page != 10 and page % 10 == 0 : #두번째 페이지 부터는 첫페이지는 a[12]
                                next_button = browser.find_element(By.XPATH,'//*[@id="section_review"]/div[3]/a[12]')
                                next_button.click()
                                # print('2번옵션')
                            #10페이지 이하     
                            elif page < 10: 
                                next_button = browser.find_element(By.XPATH,'//*[@id="section_review"]/div[3]/a[{0}]'.format( (page % 10) + 1)) #
                                next_button.click()   
                                # print('3번옵션')
                            #10 페이지 이상 
                            else :
                                next_button = browser.find_element(By.XPATH,'//*[@id="section_review"]/div[3]/a[{0}]'.format( (page % 10) + 2)) #
                                next_button.click()   
                                # print('4번옵션')
                        except:
                            pass 
                        
                        # 10페이지 이하 
                        # //*[@id="section_review"]/div[3]/a[2] 2페이지
                        # //*[@id="section_review"]/div[3]/a[3] 3페이지
                        # //*[@id="section_review"]/div[3]/a[4] 4페이지 
                        
                        # //*[@id="section_review"]/div[3]/a[10] 10페이지 
                        
                        #10 페이지 이상 
                        # //*[@id="section_review"]/div[3]/a[3] 12페이지
                        # //*[@id="section_review"]/div[3]/a[4] 13페이지 
                        # //*[@id="section_review"]/div[3]/a[10] 19페이지 
                        # //*[@id="section_review"]/div[3]/a[11] 20페이지 
                        
                        # 10페이지 단위 
                        # //*[@id="section_review"]/div[3]/a[11] 10페이지만 11이고
                        
                        # //*[@id="section_review"]/div[3]/a[12] 20페이지 부터는 12 
                        # //*[@id="section_review"]/div[3]/a[12] 30페이지 
                        # //*[@id="section_review"]/div[3]/a[12] 40페이지
                    
                    # 전체 페이지 리뷰 완료했을시 
                    ### 해당 상품 페이지 탭 닫기 
                    browser.close()  
                    browser.switch_to.window(browser.window_handles[0]) #첫번째 탭으로 돌아가기 (반드시 해야함)
                    time.sleep(random.uniform(2,3))
                else:
                    continue  # '쇼핑몰별 최저가' 페이지 구조 아닌 경우 그냥 skip  
            except : 
                #리뷰 리스트와 동일하게 에러가 나는 경우에는 에러리스트 추가 --> 나중에 문제 확인하고 수정해야 함 
                error_dict = {}
                error_dict['pdt_name'] = pdt_name 
                error_dict['pdt_page'] = pdt_page
                error_dict['pdt_num'] = pdt_num
                error_list.append(error_dict)
                
                #수집 에러가 나는 경우에도 해당 상품 페이지는 닫아야 함 
                browser.close()  
                browser.switch_to.window(browser.window_handles[0])
                time.sleep(random.uniform(2,3))
                continue 
        except NoSuchElementException:
            # '쇼핑몰별 최저가' 구조 실패한 경우 다른 xpath 구조 시도하지 않고 넘어감
            continue

    # 여러 상품 보여주는 페이지: 다음 페이지 누르기 
    # 1페이지랑 그 이후 페이지가 다름 
    wait = WebDriverWait(browser, 5)
    if pdt_page == 1: 
        next_page_button = browser.find_element(By.XPATH,'//*[@id="content"]/div[1]/div[4]/a')
        next_page_button.click()
    else:
        next_page_button = browser.find_element(By.XPATH,'//*[@id="content"]/div[1]/div[4]/a[2]')
        next_page_button.click()
            ### 여기에도 try except문 써서 continue 하게 해줘보자 
        '''
        //*[@id="content"]/div[1]/div[4]/a #1페이지 
        //*[@id="content"]/div[1]/div[4]/a[2] #2페이지부터 
        //*[@id="content"]/div[1]/div[4]/a[2]
        //*[@id="content"]/div[1]/div[4]/a[2]
        '''

<=================  리뷰 페이지 :  1  =================>
-------------------------------------------------
상품명 :  LE TROIS 23F W 이보영의 르투아 리얼 폭스퍼 롱구스다운 / 등록일 :  2023.11.
총리뷰수 :  167
마지막페이지 :  9
1 페이지
2 페이지
3 페이지
4 페이지
5 페이지
6 페이지
7 페이지
8 페이지
9 페이지
-------------------------------------------------
상품명 :  올리브데올리브 폭스퍼 후드 다운 롱 패딩 YP3WH830 / 등록일 :  2023.10.
총리뷰수 :  510
마지막페이지 :  26
1 페이지
2 페이지
3 페이지
4 페이지
5 페이지
6 페이지
7 페이지
8 페이지
9 페이지
10 페이지
11 페이지
12 페이지
13 페이지
14 페이지
15 페이지
16 페이지
17 페이지
18 페이지
19 페이지
20 페이지
21 페이지
22 페이지
23 페이지
24 페이지
25 페이지
26 페이지
-------------------------------------------------
상품명 :  폴햄키즈 공용 구스다운 롱패딩 점퍼 PKD4JP3010 / 등록일 :  2023.11.
총리뷰수 :  249
마지막페이지 :  13
1 페이지
2 페이지
3 페이지
4 페이지
5 페이지
6 페이지
7 페이지
8 페이지
9 페이지
10 페이지
11 페이지
12 페이지
13 페이지
-------------------------------------------------
상품명 :  폴햄 23FW 여성용 퀼팅 롱 경량 패딩 1종 / 등록일 :  2023.09.
총리뷰수 :  1691
마지막페이지 :  85
1 페이지
2 페이지
3 페이지
4 페이지
5 페이지
6 페이지
7 페이지
8 페이지
9 페이지
10 페이지
11 페이지
12 페이지
13 페이지
14 페이지
15 페이지
16 페이지
17 페이지
18 

In [13]:
# 우리가 아는 데이터 프레임이 나옴 
df_review = pd.json_normalize(review_list)     
df_review.loc[df_review['review'].isna()].shape
df_review = df_review.drop(df_review[df_review['review'].isna()].index)

df_error = pd.json_normalize(error_list)  

In [14]:
df_review

,pdt_name,rating,store,date,review
0,LE TROIS 23F W 이보영의 르투아 리얼 폭스퍼 롱구스다운,5,주식회사 케이티알파,23.12.05.,카키 블랙 라벤더 이렇게 결국 사버렸네요 ㅋㅋ 라벤더는 좀 안어울릴수도 있겠가 반품...
1,LE TROIS 23F W 이보영의 르투아 리얼 폭스퍼 롱구스다운,5,주식회사 케이티알파,23.12.03.,빠르게 잘받았습니다. 우연히 채널돌리다 홈쇼핑에서 보고 괜찮아보여 주문했습니다. 6...
2,LE TROIS 23F W 이보영의 르투아 리얼 폭스퍼 롱구스다운,5,주식회사 케이티알파,23.11.08.,가볍고 보온성 좋은 캐시미어 찾다가 저렴한 가격으로 구매했어요. 아주 완전 부드럽지...
3,LE TROIS 23F W 이보영의 르투아 리얼 폭스퍼 롱구스다운,5,주식회사 케이티알파,23.12.05.,"카키 사고 결국 네이비, 라벤더도 구입. 네이비능 생각했던대로 무난해요. 핏은 카키..."
4,LE TROIS 23F W 이보영의 르투아 리얼 폭스퍼 롱구스다운,5,신세계몰,23.11.18.,막스마라 스타일을 좋아하는데 비슷한 캐이프 디자인이 맘에 들어서 구매했어요. 핸드메...
...,...,...,...,...,...
6591,씨씨콜렉트 폭스퍼 벨티드 구스다운 롱 패딩 점퍼 E224PSG191,5,롯데 기흥점 씨씨콜렉트,24.01.21.,가슴부분이 작아요 평소 55입어요. 컬러는 블루는 절대 아니고 그레이민트라이트 입니다
6592,씨씨콜렉트 폭스퍼 벨티드 구스다운 롱 패딩 점퍼 E224PSG191,5,현대동대문 CC콜렉트,24.01.07.,색상찾기가 어려웠는데... 배송도 빠르고 잘 맞아요
6593,씨씨콜렉트 폭스퍼 벨티드 구스다운 롱 패딩 점퍼 E224PSG191,5,롯데 기흥점 씨씨콜렉트,23.04.06.,털이.좀빠져.이거외엔.모양도예쁘고.좋은데
6594,씨씨콜렉트 폭스퍼 벨티드 구스다운 롱 패딩 점퍼 E224PSG191,5,롯데 기흥점 씨씨콜렉트,23.03.07.,가벼우면서도 포근하고 따뜻


In [15]:
df_error

""


In [16]:
df_review.to_excel('naver_shopping_data2.xlsx')